In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
import os
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
# import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-10/Mask_ERF_iris-10.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [3]:
track = pd.read_csv('../saved-files/threshold-10/final-tracks-threshold-10-excluding-area.csv')
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ
0,0,4,112.472423,381.827530,14,10,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,4,0 days 00:00:00,133.652175,13.245867,34.567482
1,1,4,113.727184,383.443874,16,10,105,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,4,0 days 00:05:00,133.177717,12.974534,35.577388
2,2,4,114.912344,385.528075,9,10,211,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,4,0 days 00:10:00,133.000505,12.780750,34.063904
3,3,4,116.591310,387.338867,9,10,315,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,4,0 days 00:15:00,132.299723,12.708913,36.584485
4,4,4,118.277261,388.995062,5,10,415,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,4,0 days 00:20:00,131.563070,12.845599,38.092506


In [4]:
len(track)

17194

## _CALCULATE AREA_

In [6]:
area = tobac.calculate_area(track, Mask_ERF, method_area=None)
# area1 = (c['area']/1000000)
area1 = np.divide(area['area'],1000000)
area['area($km^2$)'] = area1

In [7]:
track['area($km^2$)'].max()

4513.352321761377

In [8]:
# # finding the maximum area
# z=track[track['area($km^2$)'] == 1433.9665540745268]
# z

In [9]:
tracks = track.drop(columns=['area'])
tracks

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,4,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,105,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,4,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,211,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,4,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,315,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,4,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,415,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,4,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17189,299,37,344.225457,230.927378,85,10,24599,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.404225e+06,147927.378233,68.294564,13.520861,8480,0 days 00:10:00,145.295316,15.240493,41.116443,138.320857
17190,300,40,345.289261,230.420788,113,10,24640,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.405289e+06,147420.787679,68.285155,13.507285,8480,0 days 00:15:00,146.446244,14.702386,40.106802,146.281856
17191,298,4,144.157813,260.357823,215,10,24525,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.204158e+06,177357.823188,70.094551,14.600400,8481,0 days 00:00:00,109.386079,15.301029,39.603226,383.006996
17192,299,7,145.597643,261.961765,216,10,24569,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.205598e+06,178961.765093,70.080315,14.638806,8481,0 days 00:05:00,107.283607,15.240493,41.116443,396.941301


In [10]:
# all tracks
tracks.to_csv('../saved-files/threshold-10/final-tracks-threshold-10.csv', index=False)

## _PROOF THAT WE HAVE AREA OF THIS KIND_

In [11]:
# mask_cell_361 = tobac.utils.mask_cell(Mask_ERF, 361, tracks)
# mask_cell_361

In [12]:
# convert = xr.DataArray.from_iris(mask_cell_361)
# convert

In [14]:
# %%time
# path = '/glade/u/home/noteng/work/research/data/'
# file = 'yrwms-nordic.mos.pcappi-0-dbz.noclass-clfilter-novpr-clcorr-block.laea-yrwms-1000.20200313.nc'
# data = xr.open_dataset(path+file)
# # equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# # equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# # equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
# equivalent_reflectivity_factor
# data.close()

In [15]:
# %%time

# for i in range(0, 9+1):

#     ## center points and extent of map ###
#     latA = 69.141281 #latitude of COMBLE site
#     lonA = 15.684166-1 #longitude of COMBLE site -1
#     xm, ym = 6,2.75 # extent from center point in lon and lat

#     fig, ax = plt.subplots(1,1,figsize=(10,9),subplot_kw={'projection': ccrs.Orthographic(lonA,latA)})

#     extent = (lonA-xm, lonA+xm, latA-ym, latA+ym) # define extent map
#     ax.set_extent(extent) # set extent of map
#     ax.coastlines(resolution='10m') # plot coastlines with high resolution: 10m


#     #change time to moth, day, year, hours and minutes
#     # dt = masked_values_to_xarray[4]['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array 
#     dt = convert['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array 
    

# #     #     # fill in the place where there is cloud with 1 and place with no cloud to be zero
#     cloud = np.asarray(convert[i],dtype='float')
#     invalid_cloud = np.greater(cloud,0)
#     cloud[invalid_cloud] = 1
#     invalid_cloud1 = np.less(cloud,1)
#     cloud[invalid_cloud1] = np.nan
    

#     cs=ax.pcolormesh(convert['lon'],
#                 convert['lat'], 
#                 # cloud*equivalent_reflectivity_factor.values[i],
#                 cloud*equivalent_reflectivity_factor.values[i],
#                 cmap='jet',
#                 transform=ccrs.PlateCarree(),
#                 vmin=-20,
#                 vmax=40
#                 )

#     ax.plot(lonA+1,latA, color='red', marker='*', markersize = 10.5,transform=ccrs.PlateCarree()) # plot red star at location of Andenes
#     # ax.plot(lonA,latA, color='red', marker='*', markersize = 20.5,transform=ccrs.PlateCarree()) # plot red star at location of Andenes
#     ax.set_title(f"Nordic Radar Mosaic: {dt[i]} UTC", fontweight='bold', fontsize=16, pad=11)  # title of figure


#     # Add a colorbar axis at the right of the graph
#     cbar_ax = fig.add_axes([0.85, 0.13, 0.047, 0.75]) #lbwh


#     cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
#     cbar.set_label("Reflectivity Factor [dBZ]", fontsize = 15)
    
#     fig.savefig(f'./new//{dt[i]}.png', dpi=100)
#     plt.close()



# # #     dirr = sorted(os.listdir('./mod-images//'))
# # #     if f'{dt[i]}.png' in dirr:
# # #         # print(f'{dt[i]}.png already exist')
# # #         pass
# # #         plt.close()
# # #     else:
# # #         fig.savefig(f'./mod-images//{dt[i]}.png', dpi=100)
# # #         plt.close()
# # print('\U0001f600\U0001f600\U0001f600\U0001f600EXECUTION DONE!!!\U0001f600\U0001f600\U0001f600\U0001f600')
# # gc.collect()

In [16]:
# %%time
# import glob
# # Build GIF
# imgs = sorted(glob.glob("./new/*.png"))
# with imageio.get_writer('large-area.gif', mode='I', duration=0.3) as writer:
#     for filename in imgs:
#         image = imageio.v2.imread(filename)
#         writer.append_data(image)

In [17]:
# %%time
# #Display image
# display.Image('large-area.gif', width=800)

# END 